In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import CRUD module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "test"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# Read all data from database
df = pd.DataFrame.from_records(db.read({}))

# Remove _id column if it exists
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add Grazioso Salvare's logo with URL anchor
image_filename = 'logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # Logo and Header with URL anchor
    html.Div([
        html.A(
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={'height':'150px', 'width':'auto'}),
            href='https://www.snhu.edu',
            target='_blank'
        ),
        html.Center(html.B(html.H1('Grazioso Salvare Search and Rescue Database'))),
        html.Center(html.H3('Created by Ryan Manchanthasouk')),
    ]),
    
    # Filter Options
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            style={'margin': '20px', 'padding': '10px'}
        )
    ]),
    
    html.Hr(),
    
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '180px',
            'width': '180px',
            'maxWidth': '180px',
            'whiteSpace': 'normal',
            'textAlign': 'left'
        },
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        selected_rows=[0],
    ),
    
    html.Br(),
    html.Hr(),
    
    # Charts Container
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            # Pie Chart
            html.Div(
                id='graph-id',
                className='col s12 m6',
                style={'width': '50%'}
            ),
            # Map
            html.Div(
                id='map-id',
                className='col s12 m6',
                style={'width': '50%'}
            )
        ]
    )
])

#############################################
# Callbacks / Controller
#############################################

# Callback for filtering data based on radio selection
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Queries based on exact specifications from the document
    if filter_type == 'water':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'mountain':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", 
                             "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'disaster':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", 
                             "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:  # reset
        query = {}
    
    df = pd.DataFrame.from_records(db.read(query))
    if not df.empty and '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')

# Callback for pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return []

    return [
        dcc.Graph(
            figure=px.pie(
                dff,
                names='breed',
                title='Distribution of Dog Breeds'
            )
        )
    ]

# Callback for highlighting selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Callback for map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '100%', 'height': '500px'}, 
               center=[30.75,-97.48], 
               zoom=10, 
               children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], 
                            children=[
                                dl.Tooltip(dff.iloc[row,4]),
                                dl.Popup([
                                    html.H1("Animal Name"),
                                    html.P(dff.iloc[row,9])
                                ])
                            ])
               ])
    ]

# Run the server
app.run_server(debug=True)